In [1]:
import torchvision
from torch.utils.data import DataLoader
from torch import nn

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root = "../data",train=True,transform=torchvision.transforms.ToTensor(),download=True)

test_data = torchvision.datasets.CIFAR10(root = "../data",train=False,transform=torchvision.transforms.ToTensor(),download=True)


d:\Application\Anaconda\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


Files already downloaded and verified
Files already downloaded and verified


In [2]:
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))


训练数据集的长度为：50000
测试数据集的长度为：10000


In [3]:
#  dataloader加载数据集
train_dataloader = DataLoader(train_data,batch_size = 64)
test_dataloader = DataLoader(test_data,batch_size = 64)

In [4]:
# 搭建神经网络

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.model = nn.Sequential(
            #  inpuT 3 output 32  卷积核5 stride 1 padding 2
            #  第一次卷积操作
            nn.Conv2d(3,32,5,1,2),

            #  最大池化操作 2 是kernel size
            nn.MaxPool2d(2),

            # 卷积操作 
            nn.Conv2d(32,32,5,1,2),

            nn.MaxPool2d(2),

            nn.Conv2d(32,64,5,1,2),

            nn.MaxPool2d(2),

            # 展平 将64个channel的4 x 4的图片全部展平 
            # 展平之后的结果是 6 4 * 4 * 4  一个一维向量
            nn.Flatten(),
            #  线性层
            nn.Linear(64 * 4 * 4,64),
            nn.Linear(64,10)
        )

    #  前向传播
    def forward(self,x):
        x = self.model(x)
        return x


In [5]:
import torch
# tudui = Tudui()
# #  BATCH_SIZE 64  CHANNEL 3 32 X 32
# input = torch.ones((64,3,32,32))

# output = tudui(input)
# print(output.shape)


In [6]:
#  创建网络模型
tudui = Tudui()
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.01
# 定义优化器  随机梯度下降
optimizer = torch.optim.SGD(tudui.parameters(),lr = learning_rate)

In [7]:
for data in train_dataloader:

        imgs,targets = data
        print(imgs.shape)
        print(targets.shape)
        

torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size

In [8]:

#  设置训练网络的一些参数
total_train_step = 0
# 记录测试的次数
total_test_step = 0

# 训练的轮数
epoch = 100
from torch.utils.tensorboard import SummaryWriter

#  添加tensorboard

writer = SummaryWriter("../logs_train")

for i in range(epoch):
    print("------第{}轮训练开始".format(i + 1))

    # 训练步骤开始
    tudui.train()
    for data in train_dataloader:

        imgs,targets = data

        # 64 x 10
        outputs = tudui.forward(imgs)

        #  计算损失
        loss = loss_fn(outputs,targets)

        #  将梯度清0
        optimizer.zero_grad()

        # 反向传播  计算梯度
        loss.backward()

        # 优化
        optimizer.step()

        #  统计训练次数
        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            # item 将tensor转换为一个数据类型
            print("训练次数:{},Loss:{}".format(total_train_step,loss.item()))
            #  绘制训练损失
            writer.add_scalar("train_loss",loss.item(),total_train_step)

    #  测试步骤开始
    tudui.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():

        #  取出测试数据集的数据
        for data in test_dataloader:

            #  取出数据
            imgs,targets = data
            outputs = tudui(imgs)

            loss = loss_fn(outputs,targets) # 计算损失

            #  统计测试集上面的总损失
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy


    print("整体测试集上面的Loss:{}".format(total_test_loss))
    print("整体测试及上面的正确率:{}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss",loss.item(),total_test_step)
    writer.add_scalar("test_accuracy",total_accuracy / test_data_size,total_test_step)
    total_test_step = total_test_step + 1

        
    


------第1轮训练开始
训练次数:100,Loss:2.280682325363159
训练次数:200,Loss:2.283000946044922
训练次数:300,Loss:2.223792552947998
训练次数:400,Loss:2.178748369216919
训练次数:500,Loss:2.109104633331299
训练次数:600,Loss:2.054152727127075
训练次数:700,Loss:1.9907876253128052
整体测试集上面的Loss:313.5263839960098
整体测试及上面的正确率:0.2897000014781952
------第2轮训练开始
训练次数:800,Loss:1.8798295259475708
训练次数:900,Loss:1.8610330820083618
训练次数:1000,Loss:1.9429835081100464
训练次数:1100,Loss:1.9663783311843872
训练次数:1200,Loss:1.698791742324829
训练次数:1300,Loss:1.6877349615097046
训练次数:1400,Loss:1.7799464464187622
训练次数:1500,Loss:1.854265570640564
整体测试集上面的Loss:283.68015921115875
整体测试及上面的正确率:0.3513000011444092
------第3轮训练开始
训练次数:1600,Loss:1.7859373092651367
训练次数:1700,Loss:1.64202082157135
训练次数:1800,Loss:1.9250802993774414
训练次数:1900,Loss:1.7377824783325195
训练次数:2000,Loss:1.893567442893982
训练次数:2100,Loss:1.5470895767211914
训练次数:2200,Loss:1.458203673362732
训练次数:2300,Loss:1.8062918186187744
整体测试集上面的Loss:269.29749488830566
整体测试及上面的正确率:0.37630000710487366
------第4

KeyboardInterrupt: 

In [ ]:
import torch
# if torch.cuda.is_available():
#     tudui = tudui.cuda()

print(torch.cuda.is_available())

True
